In [297]:
# Data processing tool for Nova Home Support
# IPython Notebook for testing
#
# Import Python packages
# Note: We're only using packages that come with Anaconda standard distribution: https://www.anaconda.com/ 
import pandas as pd # For importing, manipulating, and exporting data
import re # Python regular expression support
import datetime # Python datetime conversion support
from concurrent.futures import process # Asynchronous callable execution
from dateutil.easter import *
import holidays
from tkinter import * # For user interface
from tkinter.ttk import * # UI widgets
from tkinter.filedialog import askopenfile, askopenfilename, askdirectory # UI-file system interaction
from copy import deepcopy
from collections import namedtuple

Range = namedtuple('Range', ['start', 'end'])

In [298]:
# Read in the dataset
df = pd.read_excel('shift.xlsx') 
# Keep useful columns
df = df[['Service 1 Description (Code)', 
         'Service Provider','Check-In Date',
         'Check-In Time',
         'Updated Check-In Date',
         'Updated Check-In Time',
         'Check-Out Date',
         'Check-Out Time',
         'Updated Check-Out Date',
         'Updated Check-Out Time',
         'Staff Worked Duration',
         'Staff Worked Duration (Minutes)']]

In [299]:
# Remove parentheses and everything within them
df['Service 1 Description (Code)'] = df['Service 1 Description (Code)'].str.replace(r'\(.*\)', '')

# Remove prefix if it exists
prefix = 'RC-SDP-CLS-320 '
df['Service 1 Description (Code)'] = df['Service 1 Description (Code)'].apply(lambda x: x[len(prefix):] if x.startswith(prefix) else x)
df['Service 1 Description (Code)'] = df['Service 1 Description (Code)'].apply(lambda x: x.rstrip())

/var/folders/kl/0xlphzyd1450znbjp9zww9tw0000gn/T/ipykernel_80513/4018409904.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Service 1 Description (Code)'] = df['Service 1 Description (Code)'].str.replace(r'\(.*\)', '')


In [300]:
# Remove everything after " /"
df['Service Provider'] = df['Service Provider'].str.split(' /', n=1).str[0]

In [301]:
# Replace Date/Time with Updated Date/Time if the latter is not NaN
df['Check-In Date'] = df['Updated Check-In Date'].fillna(df['Check-In Date'])
df['Check-In Time'] = df['Updated Check-In Time'].fillna(df['Check-In Time'])
df['Check-Out Date'] = df['Updated Check-Out Date'].fillna(df['Check-Out Date'])
df['Check-Out Time'] = df['Updated Check-Out Time'].fillna(df['Check-Out Time'])
df.drop(['Updated Check-In Date', 'Updated Check-In Time','Updated Check-Out Date', 'Updated Check-Out Time'], axis=1, inplace=True)

In [302]:
# Create datetime series in Python format
CIDT = df['Check-In Date'].str.cat(df['Check-In Time'], sep=' ')
CODT = df['Check-Out Date'].str.cat(df['Check-Out Time'], sep=' ')
CIDT = CIDT.apply(lambda x: datetime.datetime.strptime(x, r'%m/%d/%Y %I:%M %p'))
CODT = CODT.apply(lambda x: datetime.datetime.strptime(x, r'%m/%d/%Y %I:%M %p'))
PAY_PERIOD = str(CIDT.min().date()) + ' - ' + str(CODT.max().date())
df['CIDT'] = CIDT
df['CODT'] = CODT

In [303]:
# Calculate time difference from check-in and check-out datetimes
CTD = (CODT - CIDT).dt.total_seconds() / 60
# Convert Staff Work Duration from Hour:Minutes to Minutes
SWD_min = df['Staff Worked Duration'].apply(lambda x: (int(x.split(':')[0]) * 60) + int(x.split(':')[1]))

In [304]:
# Sanity check:
# 1. Check if Staff Work Duration ==  Staff Work Duration (Minutes)
sanity1 = (SWD_min == df['Staff Worked Duration (Minutes)'])
# 2. Check if |Staff Work Duration (Minutes) - Calculated Time Difference| <= 1
sanity2 = ((df['Staff Worked Duration (Minutes)'] - CTD).abs() <= 1.1) # 1.1 to avoid float precision issues
df["Error Check"] = (sanity1 & sanity2) # The data is "sane" only when both checks are passed

In [305]:
#PAYROLL (a)
df = df.rename(columns={'Service 1 Description (Code)': 'Shift Code'})
df_payroll = deepcopy(df.loc[~df['Shift Code'].str.contains('Adaptive Behavior Treatment')])
# split the 'Service Provider' column on comma separator and extract First name and Surname
df_payroll[['Surname', 'First Name']] = df_payroll['Service Provider'].str.split(', ', expand=True)
# concatenate First name and Surname columns in the desired order
df_payroll['Service Provider'] = df_payroll['First Name'] + ' ' + df_payroll['Surname']
# drop First name and Surname columns
df_payroll.drop(columns=['First Name', 'Surname'], inplace=True)
# Now process billing:
# Read "Manger Rates" sheet
manager_rates = pd.read_excel("billing.xlsx", sheet_name="Manager Rates")
# Read "Non-manager Rates" sheet
non_manager_rates = pd.read_excel("billing.xlsx", sheet_name="Non-manager Rates")
non_manager_rates = non_manager_rates[['Shift Code', 'Regular Hourly Wage', 'BOT Calculation', 'Accrue Rate']]
#merge:
#PAYROLL (c)
df_payroll=pd.merge(df_payroll, non_manager_rates, how='left', on='Shift Code')

In [306]:
manager_rates.columns

Index(['Name', 'Non-exempt Hourly Wage', 'Exempt Weekly Wage',
       'Exempt Biweekly Wage', 'Accrue Rate'],
      dtype='object')

In [307]:
#PAYROLL(b)
for name, accrued in zip(manager_rates['Name'], manager_rates['Accrue Rate']):
    df_payroll.loc[(df_payroll['Service Provider'] == name), ['Accrue Rate']] = accrued


In [308]:
# Helper functions
def approved_holiday(YEARS):
    approved_holiday = set()
    holiday_set = set(["Thanksgiving", "Christmas Day"])
    for year in YEARS:
        ah = {k for k, v in holidays.US(years=year).items() if v in holiday_set}
        approved_holiday = approved_holiday.union(ah)
        approved_holiday.add(easter(year))
        approved_holiday.add(datetime.date(year, 12, 24))
        approved_holiday.add(datetime.date(year, 12, 31))
        approved_holiday.add(datetime.date(year-1, 12, 31))
    return approved_holiday

def approved_holiday_hours(YEARS):
    approved_holiday_dt = []
    for x in approved_holiday(YEARS):
        if (x.month == 12 and x.day == 31):
            approved_holiday_dt.append(Range(start = datetime.datetime.combine(x, datetime.time(hour=23)),end = datetime.datetime.combine(x+datetime.timedelta(days=1), datetime.time(hour=23))))
        else:
            approved_holiday_dt.append(Range(start = datetime.datetime.combine(x, datetime.time(hour=7)), end = datetime.datetime.combine(x, datetime.time(hour=23))))
    return approved_holiday_dt

def work_holiday_overlap(work_range, ahh_list):
    total_overlap = 0.0
    for ahh in ahh_list:
        latest_start = max(work_range.start, ahh.start)
        earliest_end = min(work_range.end, ahh.end)
        delta = (earliest_end - latest_start)
        overlap = max(0, delta.total_seconds())
        total_overlap += overlap
    return total_overlap/60 #in minutes

def calc_holiday_hours(df_payroll):
        years = set([min([df_payroll['CIDT'][i].year for i in range(len(df_payroll))]), max([df_payroll['CODT'][i].year for i in range(len(df_payroll))])])
        duration_range = [Range(start = df_payroll['CIDT'][i].to_pydatetime(), end = df_payroll['CODT'][i].to_pydatetime()) for i in range(len(df_payroll))]
        ahh_list = approved_holiday_hours(years)
        holiday_hours = [work_holiday_overlap(duration_range[i], ahh_list) for i in range(len(df_payroll))]
        df_payroll['Holiday Hours'] = holiday_hours
        
def is_manager(name, manager_rates):
    return name in set(manager_rates['Name'])

def manager_is_exempt(name, df_payroll):
    exempt_mins = df_payroll.loc[(df_payroll['Service Provider'] == name) & (df_payroll['Shift Code'] != 'MGR Direct Care')]['Staff Worked Duration (Minutes)'].sum()
    non_exempt_mins = df_payroll.loc[(df_payroll['Service Provider'] == name) & (df_payroll['Shift Code'] == 'MGR Direct Care')]['Staff Worked Duration (Minutes)'].sum()
    return (exempt_mins >= non_exempt_mins)

def worked_overtime(name, df_payroll):
    return df_payroll.loc[df_payroll['Service Provider'] == name]['Staff Worked Duration (Minutes)'].sum() > (40*60)


In [309]:
calc_holiday_hours(df_payroll)

In [310]:
manager_is_exempt('Mikayla Napier', df_payroll)

True

In [311]:
staff_names = df_payroll['Service Provider'].unique()
manager_status = [is_manager(i,manager_rates) for i in staff_names]

non_mgr = [] #list of names
mgr = [] #list of names
for i, name in enumerate(staff_names):
    if manager_status[i]:
        mgr.append(name)
    else:
        non_mgr.append(name)


In [312]:
#non managers
def non_manager_payroll(non_mgr, df_payroll, PAY_PERIOD):

    TGW = [] #Total Gross Wages
    THW = [] #Total Hours Worked
    THA = [] #Total Hours Accured
    
    for i, name in enumerate(non_mgr):
        df_indiv = df_payroll.loc[df_payroll['Service Provider'] == name]
        total_hours_worked = df_indiv['Staff Worked Duration (Minutes)'].sum()/60
        THW.append(total_hours_worked)
        THA.append((df_indiv['Staff Worked Duration (Minutes)']/60 * df_indiv['Accrue Rate']).sum())
        base_salary = (df_indiv['Regular Hourly Wage']*df_indiv['Staff Worked Duration (Minutes)']/60).sum()
        blended_overtime = 0
        if worked_overtime(name, df_payroll):
            BOT_base_rate = (df_indiv['BOT Calculation']*df_indiv['Staff Worked Duration (Minutes)']/60).sum()/total_hours_worked
            blended_overtime = 0.5 * BOT_base_rate * (total_hours_worked-40)
        holiday_bonus = 0.5 * (df_indiv['Holiday Hours']*df_indiv['Regular Hourly Wage']).sum() 
        TGW.append(base_salary + blended_overtime + holiday_bonus)
    return pd.DataFrame({'Employee Name': non_mgr, 'Pay Peiord': [PAY_PERIOD]*len(THA), 'Total Gross Wages': TGW, 'Total Hours Worked': THW, 'Accured Time Off':THA})
    

In [313]:
non_manager_payroll(non_mgr, df_payroll, PAY_PERIOD)

,Employee Name,Pay Peiord,Total Gross Wages,Total Hours Worked,Accured Time Off
0,Jessica Robles,2023-04-16 - 2023-04-21,299.833333,8.566667,0.342667
1,Elli Hernandez,2023-04-16 - 2023-04-21,1207.436758,47.200000,1.888000
2,Vick Duenas,2023-04-16 - 2023-04-21,206.125000,8.083333,0.323333
3,Michelle Song,2023-04-16 - 2023-04-21,631.550000,24.766667,0.990667
4,Kenneth Inocencio,2023-04-16 - 2023-04-21,470.675000,16.516667,0.660667
5,Sheena Martin,2023-04-16 - 2023-04-21,983.358333,32.516667,1.300667
6,Miriam Hernandez,2023-04-16 - 2023-04-21,222.541667,8.166667,0.326667
7,Desirae Duarte,2023-04-16 - 2023-04-21,720.025000,24.416667,0.976667
8,Chelsie Ellis,2023-04-16 - 2023-04-21,272.533333,8.516667,0.340667
9,Antonia Lema,2023-04-16 - 2023-04-21,834.295833,24.716667,0.988667


In [293]:
def manager_payroll(mgr, manager_rates, df_payroll, PAY_PERIOD):
    TGW = [] #Total Gross Wages
    THW = [] #Total Hours Worked
    THA = [] #Total Hours Accured
    Exempt = [] #Exempt status
    for name in mgr:
      df_indiv = df_payroll.loc[df_payroll['Service Provider'] == name]
      total_hours_worked = df_indiv['Staff Worked Duration (Minutes)'].sum()/60
      exempt_hours_worked = df_indiv.loc[df_indiv['Shift Code'] != 'MGR Direct Care']['Staff Worked Duration (Minutes)'].sum()/60
      non_exempt_rate = manager_rates.loc[manager_rates['Name'] == name]['Non-exempt Hourly Wage'][0]
      THW.append(total_hours_worked)
      THA.append((df_indiv['Staff Worked Duration (Minutes)']/60 * df_indiv['Accrue Rate']).sum())
      if manager_is_exempt(name, df_payroll):
        MGR_weekly_salary = manager_rates.loc[manager_rates['Name'] == name]['Exempt Weekly Wage'][0]
        if name == 'Mikayla Napier':
          MGR_weekly_salary = manager_rates.loc[manager_rates['Name'] == name]['Exempt Biweekly Wage'][0]
        non_exempt_salary = (df_indiv['Regular Hourly Wage']*(total_hours_worked - exempt_hours_worked)).sum()
        holiday_bonus = 0.5 * (df_indiv['Holiday Hours'] * non_exempt_rate).sum()
        TGW.append(max(MGR_weekly_salary, non_exempt_salary) + holiday_bonus)
        Exempt.append('Exempt')
      else: #non-exempt
        base_salary = (non_exempt_rate * df_indiv['Staff Worked Duration (Minutes)']/60).sum()
        blended_overtime = 0
        if worked_overtime(name, df_payroll):
            BOT_base_rate = non_exempt_rate
            blended_overtime = 0.5 * BOT_base_rate * (total_hours_worked - 40)
        holiday_bonus = 0.5 * (df_indiv['Holiday Hours'] * non_exempt_rate).sum()
        TGW.append(base_salary + blended_overtime + holiday_bonus)
        Exempt.append('Non-exempt')
    return pd.DataFrame({'Employee Name': mgr, 'Pay Peiord': [PAY_PERIOD]*len(THA), 'Exempt Status': Exempt, 'Total Gross Wages': TGW, 'Total Hours Worked': THW, 'Accured Time Off':THA})

    

In [294]:
manager_payroll(mgr, manager_rates, df_payroll, PAY_PERIOD)

,Employee Name,Pay Peiord,Exempt Status,Total Gross Wages,Total Hours Worked,Accured Time Off
0,Mikayla Napier,2023-04-16 - 2023-04-21,Exempt,0.0,1.116667,0.075933


In [295]:
df_indiv = df_payroll.loc[df_payroll['Service Provider']=='Mikayla Napier']

In [314]:
df_payroll

,Shift Code,Service Provider,Check-In Date,Check-In Time,Check-Out Date,Check-Out Time,Staff Worked Duration,Staff Worked Duration (Minutes),CIDT,CODT,Error Check,Regular Hourly Wage,BOT Calculation,Accrue Rate,Holiday Hours
0,HSSFXWKND,Jessica Robles,04/16/2023,02:51 pm,04/16/2023,11:25 pm,8:34,514,2023-04-16 14:51:00,2023-04-16 23:25:00,True,35.00,35.00,0.040,0.0
1,HSS2,Elli Hernandez,04/16/2023,02:54 pm,04/16/2023,11:41 pm,8:47,527,2023-04-16 14:54:00,2023-04-16 23:41:00,True,26.25,26.25,0.040,0.0
2,HSS1,Vick Duenas,04/16/2023,02:56 pm,04/16/2023,11:01 pm,8:05,485,2023-04-16 14:56:00,2023-04-16 23:01:00,True,25.50,25.50,0.040,0.0
3,HSS1,Michelle Song,04/16/2023,06:50 am,04/16/2023,02:58 pm,8:07,487,2023-04-16 06:50:00,2023-04-16 14:58:00,True,25.50,25.50,0.040,0.0
4,CCR,Kenneth Inocencio,04/16/2023,06:52 am,04/16/2023,03:07 pm,8:15,495,2023-04-16 06:52:00,2023-04-16 15:07:00,True,31.50,31.50,0.040,0.0
5,HSSFXW,Sheena Martin,04/16/2023,07:04 am,04/16/2023,03:10 pm,8:06,486,2023-04-16 07:04:00,2023-04-16 15:10:00,True,31.50,31.50,0.040,0.0
6,OA2,Miriam Hernandez,04/16/2023,10:54 pm,04/17/2023,07:04 am,8:10,490,2023-04-16 22:54:00,2023-04-17 07:04:00,True,27.25,27.25,0.040,0.0
7,IHSS Asleep,Elli Hernandez,04/16/2023,11:45 pm,04/17/2023,07:06 am,7:21,441,2023-04-16 23:45:00,2023-04-17 07:06:00,True,0.00,17.75,0.040,0.0
8,CCR,Desirae Duarte,04/17/2023,02:46 pm,04/17/2023,10:53 pm,8:06,486,2023-04-17 14:46:00,2023-04-17 22:53:00,True,31.50,31.50,0.040,0.0
9,HSSFXW,Sheena Martin,04/17/2023,02:51 pm,04/17/2023,11:01 pm,8:09,489,2023-04-17 14:51:00,2023-04-17 23:01:00,True,31.50,31.50,0.040,0.0
